In [1]:
# Install req. packages
! pip install numpy
! pip install python-midi

  Using cached https://files.pythonhosted.org/packages/6a/a9/c01a2d5f7b045f508c8cefef3b079fe8c413d05498ca0ae877cffa230564/numpy-1.14.5-cp27-cp27mu-manylinux1_x86_64.whl
  Cache entry deserialization failed, entry ignored


In [42]:
import numpy as np
import midi

np.set_printoptions(threshold=np.nan)

In [56]:
# Initialization

# MIDI file to process
filename = 'data/warpigs.mid'

## Numpy examples

CHANNELS = 16
LENGTH = 100000 # number of input MIDI songs? or number of events?

ES  = 0     # size of the full event representation
ES += 8     # event type (ES[0]: NoteOff, ES[1]: NoteOn, ES[2]: AfterTouch, ES[3]: ProgramChange, ES[4]: ControlChange, ES[5]: PitchWheel, ES[6]: ChannelAfterTouch, ES[7]: EndOfTrack)
ES += 128   # pitch
ES += 1     # velocity
ES += 1     # bend
ES += 128   # instrument

#- Create a big tensor with zeros
midi_tensor = np.zeros(shape=(LENGTH, CHANNELS, ES))


In [ ]:
#- Iterate the MIDI file contents

midi_f = midi.read_midifile(filename)
e_counter = 0
for n_track in range(len(midi_f)): # iterate over tracks
    for n_event in range(len(midi_f[n_track])): # iterate over events
        #- Create an event vector
        event = np.zeros(ES)
        event_midi = midi_f[n_track][n_event]
        if type(event_midi) == midi.events.NoteOffEvent:
            event[0] = 1
            event[8 + event_midi.pitch] = 1
            event[8 + 128] = event_midi.velocity / 127. # velocity range is 0-127            
            # print event
        elif type(event_midi) == midi.events.NoteOnEvent:
            event[1] = 1
            event[8 + event_midi.pitch] = 1
            event[8 + 128] = event_midi.velocity / 127. # velocity range is 0-127 
            # print event
        elif type(event_midi) == midi.events.AfterTouchEvent:
            event[2] = 1
            event[8 + event_midi.pitch] = 1            
            event[8 + 128] = event_midi.value / 127. # pressure range is 0-127 --- safe to store this here?
            # print event
        elif type(event_midi) == midi.events.ProgramChangeEvent:
            event[3] = 1
            event[8 + 128 + 1 + 1 + event_midi.value] = 1
            # print event
        elif type(event_midi) == midi.events.ControlChangeEvent:
            # TODO: store as velocity and pressure? Read docs to get clear picture
            event[4] = 1
            # event_midi.control
            # event_midi.value            
        elif type(event_midi) == midi.events.PitchWheelEvent:
            event[5] = 1
            event[8 + 128 + 1] = event_midi.pitch / 8192. # pitch bend range is -8192-8192
        elif type(event_midi) == midi.events.ChannelAfterTouchEvent:
            event[6] = 1
            # event_midi.value            
#         elif type(event_midi) == midi.events.EndOfTrackEvent:
#             event[7] = 1
        else:
            continue
            # print "Event of type {} unmanaged".format(type(event_midi))
        # print event
        #- Add the vector         
        midi_tensor[e_counter , event_midi.channel, :] = event
        e_counter += 1
# print midi_tensor

# Questions:
# 1. distinction tracks / channels?
# 2. length are events or midis?
# 3. slot for bend goes from [-1, 1], should it be [0,1]?
# 4. afterouch events (key and channel) have no specific slot for pressure, key using now same as velocity
# 5. end of track events have no channel... leaving out

In [22]:
# Save to a compressed tensor
np.savez_compressed('midi.npz', midi=midi_tensor)

In [23]:
# Load
midi_loaded = np.load('./midi.npz')['midi']
midi_loaded.shape

(100, 16, 266)